# 💡Bank Loan Default Risk Analysis (EDA)

## 🥇Approach
- Individual analysis of Previous Data frame
- Individual analysis of Application Data frame
- Analysis of MERGED Data Frame

## Table of Content

### 🎧Data understanding
- Importing various libraries
- Read the data
- Analyze the shape
- Analyze the data type

### ✂Data Cleaning and Manipulation
- Check for null value
- Check for null percentage if required
- Deal with null values.----->Either drop the columns or impute the missing value.

### 🛒Structuring
- Check for number of unique value in each column
- Clubbing the columns in similar bucket depending on its nature
  - Categorical
  - Continuous 
  - Unique ID type
  
### 🔧Data analysis
- Performed univariate and Bivariate analysis on variables.
- Pearson's Correlation Coefficent.

### 🖇Merge the Data frame via Inner Join
- Analysis

##### 💼Main Variables used in the Assignment
- df = Previous _application data set is stored in df
- df1 = Application data set is stored in df1
- categoricaldf = Lists of categorical columns in df
- continuousdf = Lists of continuous(Numeric) columns in df
- idnodf = Lists of unique identifiers columns in df
- categoricald1f = Lists of categorical columns in df
- continuousdf1 = Lists of continuous(Numeric) columns in df
- idnodf1 = Lists of unique identifiers columns in df
- Defaulter_df1 = Store defaulter of Current application
- Non_defaulter_df1 = Store Non defaulter of Current application
- Defaulter = Store defaulter of Merged data frame
- Non_defaulter = Store Non defaulter of Merged data frame
##### All other variables which are not mentioned above and used in the code are described in comments

### User defined function used in EDA

1. convert_to_percentage()
    - The function is used to convert the count value on axis in countplot to their respective percentage value without hue parameter in it.
2. convert_to_percentage_hue()
    - The function is used to convert the count value on axis in countplot to their respective percentage value with hue parameter in it.

In [ ]:
# 1.This func is called when hue parameter is not used in count plot (univariate analysis is done)
# x is the column which is passed ,step is the no. of division of the y axis
def convert_to_percentage(x,step=10):
    upper_l=x.value_counts().max() # max value in colunm to properly use area of graph 
    total_l=len(x) #tatal count in the column to find the percentage
    # Dividing the y axis depending on the value of step default value=10
    ticks=np.arange(0,upper_l,upper_l//step)
    labels=["{}%".format(i*100//total_l)for i in ticks] # Changing counts to it's percentage
    return (plt.yticks(ticks,labels))

# 2.This  func is called when hue parameter is used in count plot
# All variables are same as above
def convert_to_percentage_hue(upper_l,total_l,step=10):
    ticks=np.arange(0,upper_l,upper_l//step)
    labels=["{}%".format(i*100//total_l)for i in ticks]
    return (plt.yticks(ticks,labels))

#### 3.Creating a user defined function top10_Corr() to find top 10 Pearson Correlation Coefficient
 - Concepts used in the creating top10_Corr()
   - Finding the pearson coefficent matrix
   - Dropping ID columns.
   - Deleting the duplicate pairs from the matrix
   - We can either retain the lower or the upper triangle values.
     - Retaining the upper triangle values
   - Converting rest to Nan so that unique pairs exists and converting the matrix to 1-D array.
   - Droping the NaN values
   - Sorting the 1-D array (Correlation coefficents)
   - Plotting and printing top 10 correlatiopn pairs.

In [ ]:
def top10_Corr(y):
    corr_matrix=y.corr(method="pearson") # Finding the pearson coefficent matrix.
    
    
    if ("SK_ID_PREV") in(y.columns):
        corr_matrix.drop(columns=["SK_ID_CURR","SK_ID_PREV"],inplace=True)# droping the column
        corr_matrix.drop(labels=["SK_ID_CURR","SK_ID_PREV"],inplace=True) # droping the rows
        
    elif ("SK_ID_CURR_Current") in(y.columns):
        corr_matrix.drop(columns=["SK_ID_CURR","SK_ID_PREV","TARGET"],inplace=True)# droping the column
        corr_matrix.drop(labels=["SK_ID_CURR","SK_ID_PREV","TARGET"],inplace=True) # droping the rows
        
    else:
        corr_matrix.drop(columns=["SK_ID_CURR","TARGET"],inplace=True)# droping the column
        corr_matrix.drop(labels=["SK_ID_CURR","TARGET"],inplace=True) # droping the rows
        
        
# Retaining the upper triangle values, converting rest to Nan so that unique pairs exists and converting the matrix to 1-D array.
    corr_matrix_uptri=corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool)).unstack()
    

# Droping the NaN values.
    corr_matrix_uptri.dropna(inplace=True)

# Sorting the 1-D array (Correlation coefficents) and showing the Top 10 Correlation coefficents.
    corr_matrix_unstack=corr_matrix_uptri.sort_values(ascending=False)
    
# Collecting top10
    top10=corr_matrix_unstack.head(10)
# Converting to the data frame
    top10=top10.reset_index()
# Reseting the columns name to some meaning name.
    top10.rename(columns={"level_0":"column_1","level_1":"column_2",0:"correlation"},inplace=True)
     
# Plotting just for visulization
    def top10_plot(x):
        x.plot()
        plt.xticks(rotation=90)
        plt.title("Top10 correlation graph")
        plt.show()
       
    return(top10,top10_plot(top10))

In [ ]:
import pandas as pd,numpy as np,matplotlib.pyplot as plt,seaborn as sns
# warning library is imported to ignore the warning in the code
import warnings
warnings.filterwarnings(action="ignore")

# Previous _application

📙***Previous _application data set contains information about the clients' previous loan data.It contains the data on whether the previous application has been Approved,Cancelled,Refused or Unused Offer.***

## Data Understanding

In [ ]:
#Read previous_application csv in variable df.
df=pd.read_csv("previous_application.csv")

# Changing the theme just for asthectic purpose.
plt.style.use("fivethirtyeight")

df.head() # Top 5  rows will be displayed.

#### 👀Getting an idea about the previous_application data set.

In [ ]:
# Analyze the shape of the data set
df.shape

In [ ]:
# Analyze data type and null value presence in each column.
df.info() 

#### 🎯 Trying to figure out ,what to do with the null values in the data set if any.

In [ ]:
# Check for the null values count in each column 
df.isnull().sum()

In [ ]:
# Check for the null percentage in each column to better take a call.
round(df.isnull().sum()/df.shape[0],3)*100

## Data Cleaning

#### Dealing with the null values

In [ ]:
# Just drop the columns with more than 40% of null values
df.drop(["AMT_DOWN_PAYMENT","RATE_DOWN_PAYMENT","RATE_INTEREST_PRIMARY","RATE_INTEREST_PRIVILEGED",
        "NAME_TYPE_SUITE","DAYS_FIRST_DRAWING","DAYS_FIRST_DUE","DAYS_LAST_DUE_1ST_VERSION",
        "DAYS_LAST_DUE","DAYS_TERMINATION","NFLAG_INSURED_ON_APPROVAL"],axis=1,inplace=True)

#### Let's Impute the continuous variable missing value in each column now with their median.

In [ ]:
# Fillna() is used to impute NaN value with median for numeric columns.

df.AMT_ANNUITY.fillna(df.AMT_ANNUITY.median(),inplace=True)

df.AMT_GOODS_PRICE.fillna(df.AMT_GOODS_PRICE.median(),inplace=True)

df.CNT_PAYMENT.fillna(df.CNT_PAYMENT.median(),inplace=True)

In [ ]:
# Fillna() is used to impute NaN value with mode for categorical columns.

df.PRODUCT_COMBINATION.fillna(df.PRODUCT_COMBINATION.mode()[0],inplace=True)

In [ ]:
# Only 1 NaN value present in AMT_CREDIT so just dropped it.

df=df[~df.AMT_CREDIT.isnull()]

In [ ]:
# Sanity Check
df.isnull().sum()

In [ ]:
df.describe()

#### 💡From the above matrix it can be clearly understood that 
- There is a huge difference in the max and 75th percentile of AMT_APPLICATION , AMT_CREDIT , AMT_GOODS_PRICE , CNT_PAYMENT ,etc which is a clear indication of the outliers.
- Also some negative value have occured in few columns.
##### We will deal with them in further analysis.

## 🛒Structuring (Clubbing the columns in bins of similar nature).

#### Trying to figure out the nature of each column and thus clubbing similar nature columns in lists.

In [ ]:
# Checking for the unique values in each column
for i in df.columns:
    print(i,"->",df[i].nunique(),df[i].dtype)

In [ ]:
df.shape[1]

In [ ]:
categoricaldf=["NAME_CONTRACT_TYPE","WEEKDAY_APPR_PROCESS_START","HOUR_APPR_PROCESS_START",
             "FLAG_LAST_APPL_PER_CONTRACT","NFLAG_LAST_APPL_IN_DAY","NAME_CASH_LOAN_PURPOSE",
             "NAME_CONTRACT_STATUS","NAME_PAYMENT_TYPE","CODE_REJECT_REASON","NAME_CLIENT_TYPE",
             "NAME_GOODS_CATEGORY","NAME_PORTFOLIO","NAME_PRODUCT_TYPE","CHANNEL_TYPE",
             "NAME_SELLER_INDUSTRY","NAME_YIELD_GROUP","PRODUCT_COMBINATION"]
continuousdf=["AMT_ANNUITY","AMT_APPLICATION","AMT_CREDIT","AMT_GOODS_PRICE","DAYS_DECISION",
            "SELLERPLACE_AREA","CNT_PAYMENT"]
idnodf=["SK_ID_PREV","SK_ID_CURR"]
# Just doing a sanity check on total numbers of columns if I have missed any.
len(categoricaldf)+len(continuousdf)+len(idnodf)

# Continuous Variable Understanding

## Univariate Analysis to deal with Outliers

In [ ]:
# Let's plot two types of plot for better understanding.
for i in continuousdf:
    print(i)
    plt.figure(figsize=[15,4]) #Increasing the size of the plots
    
    plt.subplot(121) # First plot code
    sns.boxplot(df[i])
    
    plt.subplot(122) # Second plot code
    sns.distplot(df[i])
    
    plt.show()

#### Points to be collected from above plots-
- The above graphs are the strong proof of the Outliers presence.
- From the DAYS_DECISION boxplot it can be clearly visualized that the negative day is somewhat logically not sound so we will convert it to positive value.

In [ ]:
# Correcting the negative values in DAYS_DECISION,SELLERPLACE_AREA columns.

df.DAYS_DECISION=df.DAYS_DECISION.abs()

df.SELLERPLACE_AREA=df.SELLERPLACE_AREA.abs()

#### Outliers Found in Continous Variable
- Fixing Continous Variable Outliers.

In [ ]:
# Capping and flooring the outliers so that justice can be done to the analysis.
for i in continuousdf:
    
    q1=np.percentile(df[i],25) # Finding the 25th percentile
    q3=np.percentile(df[i],75) # Finding the 75th percentile
    
    iqr=q3-q1 # Finding Inter Quatile Range
    
    uw=q3+1.5*iqr # Finding the upper whisker limit
    lw=q1-1.5*iqr # Finding the lower whisker limit
    
    df[i]=np.where(df[i]<lw,lw,df[i]) # replacing the lower boundary outliers with the lower whisker value
    df[i]=np.where(df[i]>uw,uw,df[i]) # replacing the upper boundary outliers with the upper whisker value

In [ ]:
# Sanity Check
for i in continuousdf:
    print(i)
    plt.figure(figsize=[11,4]) #Increasing the size of the plots
    
    plt.subplot(121) # First plot code
    sns.boxplot(df[i])
    
    plt.subplot(122) # Second plot code
    sns.distplot(df[i])
    
    plt.show()

# Bivariate Analysis

In [ ]:
# Let's  plot two continuous variables.

for i in continuousdf: # Automating all the continuous variables pair scatterplot
    for j in continuousdf:
        if i!=j: # Condition to avoid same variable on x and y axis
            
            print(i,"VS",j)
            sns.scatterplot(df[i],df[j])
            plt.show()

In [ ]:
(top10_Corr(df)) # Top10 previous application Correlation

# Categorical Variable

### Univariate Analysis

In [ ]:
for i in categoricaldf:# Automating all the categorical variables countplot
    print(i)
    
    plt.figure(figsize=[9,6])
    sns.countplot(df[i])
    convert_to_percentage(df[i])
    plt.xticks(rotation=90)
    
    plt.show()

#### Assumption
- In NAME_CONTRACT_TYPE ,NAME_CLIENT_TYPE columns I am assuming XNA as missing category and imputing it with the column's mode.

In [ ]:
# Imputing NAME_CONTRACT_TYPE with mode
df.NAME_CONTRACT_TYPE=df.NAME_CONTRACT_TYPE.replace("XNA",df.NAME_CONTRACT_TYPE.mode()[0])
# Imputing NAME_CLIENT_TYPE with mode
df.NAME_CLIENT_TYPE=df.NAME_CLIENT_TYPE.replace("XNA",df.NAME_CLIENT_TYPE.mode()[0])

In [ ]:
#Ok ,let's correct the Cancelled spelling
df.NAME_CONTRACT_STATUS=df.NAME_CONTRACT_STATUS.apply(lambda x:x.replace("Canceled","Cancelled"))

In [ ]:
# Sanity Check
df.NAME_CONTRACT_STATUS.value_counts().plot.pie(autopct="%0.01f%%")
plt.title("Previous CONTRACT STATUS")
plt.ylabel("")
plt.show()

In [ ]:
# Sanity Check
df.NAME_CLIENT_TYPE.value_counts().plot.pie(autopct="%0.01f%%",explode=[0,0.1,0])
plt.title("Overall Previous CLIENT TYPE")
plt.ylabel("")
plt.show()
print("The above plot reveals that previously the most loan applications were from repeater.")

##### We need to have keen eye on refused client when they come next time for loan.So let's store the refused client information in seprate variable for future use if any.

In [ ]:
# storing Refused Client details in seprate variable.
RefusedClient=df[df.NAME_CONTRACT_STATUS=="Refused"]

RefusedClient.NAME_CLIENT_TYPE.value_counts().plot.pie(autopct="%0.01f%%")
plt.title("Refused Client Type ")
plt.ylabel("")
plt.show()
print("🙄Among the refused application previously maximum were repeater.")

In [ ]:
(RefusedClient[RefusedClient.NAME_CLIENT_TYPE=="Repeater"]["CODE_REJECT_REASON"].value_counts(normalize=True)*100).plot.barh()
plt.title("Refused Repeater CLIENT's Reason and its count\n")
plt.xlabel("Percentage")
plt.show()
print("The reason for previously refused application of repeater clients are maximum due to HC.")

#### Assumption
- As the meaning of XAP,XNA in CODE_REJECT_REASON column is not clearly mentioned so I am assuming them as a valid category for this column..
- I have decided not to impute them as there presence are huge in the column.

### 🍁Few points which have poped out from the above plots for previous data set are-
- Cash and Consumer loans seem to be two most popular category asked by clients.
- The maximum numbers of the applications were filled during 9am to 4pm.
- Purpose for most of the loans are for XAP and XNA.
- Most of the clients applications seem to be Approved.
- Payment method that client chose to pay for the previous application is maximum for Cash through the bank.
- Maximum previous application rejection reasons was XAP.
- Most of the clients applying seems to be Repeater.
- Top 5 goods which clients applied in the previous applications are...
  - XNA                       
  - Mobile                      
  - Consumer Electronics        
  - Computers                   
  - Audio/Video
- Maximum previous application  was for POS followed by Cash.
- Credit and cash offices seem to be popular channel of acquiring the clients followed by country-wide.
- Top 3 industry of the seller are...
  - XNA                    
  - Consumer electronics
  - Connectivity 
- Most of the grouped interest lies in the category other than low_action.
- The reason for previously refused application are of repeater client is maximum due to HC.

# Categorical-Continuous Analysis

#### Let's try to analyze the interdependence between two variables.

In [ ]:
for i in categoricaldf: # Automating categorical- continuous plot
    for j in continuousdf:
        
        print(i,"Vs",j)
        plt.figure(figsize=[15,4])
        
        plt.subplot(121)
        sns.boxplot(df[i],df[j])# Plot 1
        plt.xticks(rotation=90)
        
        plt.subplot(122)
        sns.barplot(df[i],df[j],ci=None,estimator=np.median)# plot 2
        plt.xticks(rotation=90)
        
        plt.show()

#### Conclusion
- The average amount ask for cash loan is comparetively greater as a result average annuity amount is also increasing for the same.
- The highest median annuites amounts ie., greater than 25000 are for-----
   - Buying a new car/home
   - Buying a used car
   - Building a house
   - Buying a garage
   - Buisness developements
   - Payments of other loans
- A trend is seen that higher the loan amount highter is the term of credit,ie., more time to pay back.
_ A very interesting insight is that clients with higher ask are refused or in other words most of the refused clients ask were greater than 15 lakh.
- Repeater seems to have greater annunity amount than the new clients even though the application amounts are approximately same.
- Bank seems to give more amount to repeater clients than initally asked for but in case of new clients it is not the pattern.
- House construction goods category is having huge amount annuity.
- Clients asked maximun amount for car portfolio.
- Most of the application's amounts are from Car dealer and corporate sales channel.

#### 🛴 Let's move to the other data set for better picture about the analysis. So we will leave the present dataset for the timing and switch to other for more clearity

# Application_data

📙***Application_data csv contains all the information of the clients at the time of application(Data is about if client has payment difficulties)***

## Data Understanding

In [ ]:
#Read application_data csv in variable df1.

df1=pd.read_csv("application_data.csv")
pd.set_option("display.max_columns",122) # For viewing  all columns in the data set
df1.head()

#### 👀Getting an idea about the application_data csv.

In [ ]:
# Analyze the shape of the data set
df1.shape

In [ ]:
df1.describe()

In [ ]:
# Analyze data type of each column 

df1.info(verbose=True)

## 🗑Data Cleaning

#### 🎯 Trying to figure out ,what to do with the null values in the data set if any.

In [ ]:
# Code to see all the rows in the next line code's O/P
pd.set_option("display.max_rows",122) 

In [ ]:
# Check for the null values in each column 
df1.isnull().sum()

In [ ]:
# Check for the percentage of null values in each column and storing the column name with null % in variable p.
null_percentage=round(df1.isnull().sum()/df1.shape[0],3)*100
null_percentage

 #### Droping the columns with >= 40% of missing values in each column.
 - As we have around 50 columns to drop so instead of droping manually, a good approach is to filter the columns to be dropped in a new variable.

In [ ]:
#Extract columns with missing values >=40% .
null_percentage[null_percentage>=40]

In [ ]:
# Drop the lists of missing column stored in (p[p>=40].index)
df1.drop(columns=null_percentage[null_percentage>=40].index).shape

#### Now as I am sure that the above code dropped the required columns perfectly by looking at the shape of the data set,So now I will add "inplace = True" Clause in the above Code.

In [ ]:
df1.drop(columns=null_percentage[null_percentage>=40].index,inplace=True)

#### As a sanity check let's again check for the columns with >=40% of missing value 

In [ ]:
round(df1.isnull().mean()*100,5)

#### We will just leave missing values with 13.5% of last 6 columns from AMT_REQ_CREDIT_BUREAU_HOUR to AMT_REQ_CREDIT_BUREAU_YEAR as these will remain untouched in our analysis

#### 😎Yup good to go now we will fix rest of the missing values in other columns. Let's fix them individually now.

#### First let's drop the rows with very less missing values i.e.,<=1% associated with respective Columns.

In [ ]:
df1=df1[~df1.AMT_ANNUITY.isnull()]
df1=df1[~df1.CNT_FAM_MEMBERS.isnull()]
df1=df1[~df1.DAYS_LAST_PHONE_CHANGE.isnull()]
df1=df1[~df1.NAME_TYPE_SUITE.isnull()]
df1=df1[~df1.EXT_SOURCE_2.isnull()]
df1=df1[~df1.OBS_30_CNT_SOCIAL_CIRCLE.isnull()]
df1=df1[~df1.DEF_30_CNT_SOCIAL_CIRCLE.isnull()]
df1=df1[~df1.OBS_60_CNT_SOCIAL_CIRCLE.isnull()]
df1=df1[~df1.DEF_60_CNT_SOCIAL_CIRCLE.isnull()]

In [ ]:
# Calculating the unique value in each column to determine the nature of the columns.

for i in df1.columns:
    print(i,"-",df1[i].nunique(),"-",df1[i].dtype)

## 🛒Structuring (Clubbing the columns in bins of similar nature).

#### Trying to figure out the nature of each column and thus clubbing similar nature columns in lists.

#### We will club similar nature columns and the just take the columns which are important in our analysis from the understanding of data dictionary for this data set.

In [ ]:
# We will keep TARGET column seprate from this clubbing in any list.

categoricaldf1=["NAME_CONTRACT_TYPE","CODE_GENDER","FLAG_OWN_CAR","FLAG_OWN_REALTY","CNT_CHILDREN","NAME_TYPE_SUITE",
                "NAME_INCOME_TYPE","NAME_EDUCATION_TYPE","NAME_FAMILY_STATUS","NAME_HOUSING_TYPE","FLAG_MOBIL","FLAG_EMP_PHONE",
                "FLAG_WORK_PHONE","FLAG_CONT_MOBILE","FLAG_PHONE","FLAG_EMAIL","OCCUPATION_TYPE","CNT_FAM_MEMBERS",
                "REGION_RATING_CLIENT","REGION_RATING_CLIENT_W_CITY","WEEKDAY_APPR_PROCESS_START","REG_REGION_NOT_LIVE_REGION",
               "REG_REGION_NOT_WORK_REGION","LIVE_REGION_NOT_WORK_REGION","REG_CITY_NOT_LIVE_CITY","REG_CITY_NOT_WORK_CITY",
                "LIVE_CITY_NOT_WORK_CITY","ORGANIZATION_TYPE","DEF_30_CNT_SOCIAL_CIRCLE","DEF_60_CNT_SOCIAL_CIRCLE"]

continuousdf1=["AMT_INCOME_TOTAL","AMT_CREDIT","AMT_ANNUITY","AMT_GOODS_PRICE","REGION_POPULATION_RELATIVE","DAYS_BIRTH",
               "DAYS_EMPLOYED","DAYS_REGISTRATION","DAYS_ID_PUBLISH","HOUR_APPR_PROCESS_START","OBS_30_CNT_SOCIAL_CIRCLE",
               "OBS_60_CNT_SOCIAL_CIRCLE","DAYS_LAST_PHONE_CHANGE"]

idnodf1=["SK_ID_CURR"]


## Data understanding and converting it's type to suitable dtype

#### Categorical Variable understanding

In [ ]:
for i in categoricaldf1:
    print(i)
    print(df1[i].unique(),df1[i].dtype,"\n")

In [ ]:
# Changing the data type of required columns.

df1.CNT_FAM_MEMBERS=df1.CNT_FAM_MEMBERS.astype(int)

df1.DEF_30_CNT_SOCIAL_CIRCLE=df1.DEF_30_CNT_SOCIAL_CIRCLE.astype(int)

df1.DEF_60_CNT_SOCIAL_CIRCLE=df1.DEF_60_CNT_SOCIAL_CIRCLE.astype(int)

In [ ]:
# Drop XNA rows from CODE_GENDER column as it is very small in number.
df1=df1[~(df1.CODE_GENDER=="XNA")]

##### In OCCUPATION_TYPE column we have NAN value Let's check it.

In [ ]:
df1.OCCUPATION_TYPE.isnull().mean()*100

##### So we have 31% missing value which is relatively high so in this column so we will just add a new category "Missing" for the NaN value.

In [ ]:
df1.OCCUPATION_TYPE.fillna("Missing",inplace=True)

# Data Imbalance

In [ ]:
(df1.TARGET.value_counts(normalize=True)*100).plot.pie(autopct="%0.1f%%")
print("The below distribution of TARGET column says that the Non defaulters are very high in number ie.,most of  the clients pay back the loan.")
plt.show()

In [ ]:
sns.countplot(df1.TARGET,hue=df1.CODE_GENDER)
print("The below distribution of TARGET column says that the Non defaulters are very high in number ie.,most of the clients pay back the loan.")
convert_to_percentage(df1.TARGET) # Calling the user defined func
plt.show()

#### 😎Ok now all categorical related columns have been successfully processed for the analysis Let's move to Continuous Variable.

## Continuous variable understanding

### Univariate Analysis  to deal with Outliers

In [ ]:
# Ploting for the continuous variable.
for i in continuousdf1:
    print(i)
    
    plt.figure(figsize=[15,4])# Increasing the plot size
    
    plt.subplot(121)  # plot 1 code
    sns.boxplot(df1[i])
    
    plt.subplot(122)  #Plot 2 code
    sns.distplot(df1[i])
    
    plt.show()

## 🤷‍♂️Outliers and some data anamolies found!!!!!!

### From the boxplot few more points have lighten up🙄What can be done!!!!
#### 👀There are some anamolies as we can clearly see that some columns related to days and dates have some negative value which are not logical,so let's first correct it.
##### 💡 In order to convert the negative term we can either use the lambda () with apply or multiply the column by (-1) or simply we can use abs()

In [ ]:
df1.DAYS_BIRTH=(df1.DAYS_BIRTH)*-1

In [ ]:
df1.DAYS_REGISTRATION=df1.DAYS_REGISTRATION.abs()

In [ ]:
df1.DAYS_ID_PUBLISH=df1.DAYS_ID_PUBLISH.abs()

In [ ]:
df1.DAYS_LAST_PHONE_CHANGE=df1.DAYS_LAST_PHONE_CHANGE.abs()

In [ ]:
df1.DAYS_EMPLOYED=df1.DAYS_EMPLOYED.abs()

#### Let's convert DAYS_BIRTH column from days to years in order to get the better picture and also change it's name to YEARS_BIRTH.

In [ ]:
# Converting age in days to age in years for better picture.

df1.DAYS_BIRTH=df1.DAYS_BIRTH//365
df1.rename(columns={"DAYS_BIRTH":"YEARS_BIRTH"},inplace=True) # Renaming column name

In [ ]:
#Sanity Check
df1.YEARS_BIRTH.unique()

In [ ]:
# Changing the name of the column in continuousdf1 list as the age in days were converted to age in years.

continuousdf1=list(map(lambda x: x.replace("DAYS_BIRTH", "YEARS_BIRTH"),continuousdf1))

#### Fixing Continous Variable Outliers.

In [ ]:
# Capping and flooring the outliers so that justice can be done to the analysis.

for i in continuousdf1:
    
    q1=np.percentile(df1[i],25) # Finding the 25th percentile
    q3=np.percentile(df1[i],75) # # Finding the 75th percentile
    
    iqr=q3-q1  # Inter Quantile Range
    
    uw=q3+1.5*iqr  # Upper whisker limit
    lw=q1-1.5*iqr  # Lower whisker limit
    
    df1[i]=np.where(df1[i]<lw,lw,df1[i])# Imputing the outliers present beyond the lower boundary with the lower whisker value
    df1[i]=np.where(df1[i]>uw,uw,df1[i])# # Imputing the outliers present beyond the upper boundary with the upper whisker value

#### Sanity Check

In [ ]:
for i in continuousdf1:
    print(i)
    
    plt.figure(figsize=[15,4])# Increasing the plot size
    
    plt.subplot(121)  # plot 1 code
    sns.boxplot(df1[i])
    
    plt.subplot(122)  #Plot 2 code
    sns.distplot(df1[i])
    
    plt.show()


# Bivariate Analysis

## Continuous-Continuous Analysis

In [ ]:
for i in continuousdf1:
    for j in continuousdf1:
        if i!=j: # Condition to avoid same variable  on both x and y axis
            
            print(i,"Vs",j)
            sns.scatterplot(df1[i],df1[j],hue=df1.CODE_GENDER)
            
            plt.show()

#### Conclusion
- Good pattern is seen between AMT_CREDIT Vs AMT_GOODS_PRICE which implies that for consumer loans , price of the goods asked is credited most of the time to the clients. 
  - Other point is that bank is not partial in terms of gender while giving the goods loan as spread of red     and blue dots are evenly plotted in the direction.

#### Top 10 correlations of df1 data set.

In [ ]:
(top10_Corr(df1)) # Top 10 Current application Correlation


## Categorical Variable

### Univariate Analysis

In [ ]:
for i in categoricaldf1:
    print(i)
    plt.figure(figsize=[15,4])
    sns.countplot(df1[i])
    convert_to_percentage(df1[i])
    plt.xticks(rotation=90)
    plt.show()

### Let's jot down points from the above Univariate analysis
- Most of the loan asked are of cash type.
- Female seems to apply more for loan.
- Most of the applicant does not own car.
- Most of the applicant seems to have their own house.
- Most of the applicants seems to have no children yet , this is just a early conclusion.
- Most of the applicants were not accompained by anyone while applying for the loan.
- Secondary/secondary special level education clients seems to be fairly high.
- Married category is high.
- Highest number of applicants seems to have 2 members in their family.
- Business Entity Type 3 and 2 are maximum to apply.
- Most of the clients for loan seems to be Laborer.
- 73 % of clients are from 2nd rating Region and city.

## Bivariate Analysis

### Categorical-Categorical Analysis

#### Categorical Vs Gender Analysis

In [ ]:
for i in categoricaldf1:
    if i!= "CODE_GENDER":
        print(i)
        sns.countplot(df1[i],hue=df1.CODE_GENDER)
        upper_l=df1.groupby(i)["CODE_GENDER"].value_counts().max()
        total_l=df1.groupby(i)["CODE_GENDER"].value_counts().sum()
        convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert y label to %
        plt.xticks(rotation=90)
        plt.show()

#### Let's collect the points from above analysis
- Among Pensioner female have applied more for loan.
- Married clients are more attracted toward loan.

### Categorical-Continuous Analysis

#### Let's try to analyze the interdependence between two variables.

In [ ]:
for i in categoricaldf1:
    for j in continuousdf1:
        
        print(i,"Vs",j)
        plt.figure(figsize=[15,4])
        
        plt.subplot(121)
        sns.boxplot(df1[i],df1[j])#Plot 1
        plt.xticks(rotation=90)
        
        plt.subplot(122)
        sns.barplot(df1[i],df1[j],ci=None)#Plot 2
        plt.xticks(rotation=90)
        
        plt.show()

### In our further analysis we will analyze with respect to the Target Variable.
-  So let's just club all the data on the basis of defaulter and non-defaulter and analyze the pattern between them.

### Univariate Segmented Analysis with respect to Target column

In [ ]:
#Storing Non defaulter data
Non_defaulter_df1=df1[df1.TARGET==0]

#Storing defaulter data
Defaulter_df1=df1[df1.TARGET==1]

### TARGET Variable (Defaulter Data VS Non Defaulter) analysis  Gender

In [ ]:
for i in categoricaldf1:
    if i!= "CODE_GENDER":
        
        plt.figure(figsize=[18,6])
        print(i)
        
        plt.subplot(121) # Code for Plot 1
        sns.countplot(Defaulter_df1[i],hue=df1.CODE_GENDER,order=Defaulter_df1[i].value_counts().index)
        plt.title("Defaulter (1)")
        upper_l=Defaulter_df1.groupby(i)["CODE_GENDER"].value_counts().max()
        total_l=Defaulter_df1.groupby(i)["CODE_GENDER"].value_counts().sum()
        convert_to_percentage_hue(upper_l,total_l) # Func call to change value to percentage
        plt.xticks(rotation=90)
        
        plt.subplot(122) # Code for Plot 2
        sns.countplot(Non_defaulter_df1[i],hue=df1.CODE_GENDER,order=Non_defaulter_df1[i].value_counts().index)
        plt.title("Non defaulter (0)")
        upper_l=Non_defaulter_df1.groupby(i)["CODE_GENDER"].value_counts().max()
        total_l=Non_defaulter_df1.groupby(i)["CODE_GENDER"].value_counts().sum()
        convert_to_percentage_hue(upper_l,total_l)  # Func call to change value to percentage
        plt.xticks(rotation=90)
        
        plt.show()

### Let's jot down the new remarkable points if any from the above  analysis grouped by Gender(0 vs 1).
- Female clients seems to have more defaulter.
- Most of the defaulters own their house.
- Majority of defaulters have no child.


#### As ORGANIZATION_TYPE plot is not clearly visible so let's analyze it separately.

### 💡Which ORGANIZATION TYPE seems to have more defaulters?

In [ ]:
plt.figure(figsize=[18,6])

sns.countplot(Defaulter_df1.ORGANIZATION_TYPE,order=Defaulter_df1.ORGANIZATION_TYPE.value_counts().index)
plt.title("Defaulter (1)")
plt.ylabel("No.of Client %")
convert_to_percentage(Defaulter_df1.ORGANIZATION_TYPE)  # Calling the user defined func to convert y label to %
plt.xticks(rotation=90)

plt.figure(figsize=[18,6])
sns.countplot(Non_defaulter_df1.ORGANIZATION_TYPE,order=Non_defaulter_df1.ORGANIZATION_TYPE.value_counts().index)
plt.title("Non defaulter (0)")
plt.ylabel("No.of Client %")
convert_to_percentage(Non_defaulter_df1.ORGANIZATION_TYPE) # Calling the user defined func to convert y label to %
plt.xticks(rotation=90)

plt.show()

### Note
- XNA category count is comparatively high and also it's meaning is not explained in the data frame  so I have not impuated it.

#### Conclusion
- Maximum clients are from "Business Entity Type 3" and most of the defaulters(25%) are also from this category.

## 💡 What is the effect of education vs income type on defaulters?

In [ ]:
plt.figure(figsize=[18,6])

plt.subplot(121)
sns.countplot(Defaulter_df1.NAME_INCOME_TYPE,hue=df1.NAME_EDUCATION_TYPE,order=Defaulter_df1.NAME_INCOME_TYPE.value_counts().index)
plt.title("Defaulter (1)")
plt.ylabel("No.of Client %")
upper_l=Defaulter_df1.groupby("NAME_INCOME_TYPE")["NAME_EDUCATION_TYPE"].value_counts().max()
total_l=Defaulter_df1.groupby("NAME_INCOME_TYPE")["NAME_EDUCATION_TYPE"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert y label to %
plt.xticks(rotation=30)
plt.legend(loc=1)
        
plt.subplot(122)
sns.countplot(Non_defaulter_df1.NAME_INCOME_TYPE,hue=df1.NAME_EDUCATION_TYPE,order=Non_defaulter_df1.NAME_INCOME_TYPE.value_counts().index)
plt.title("Non defaulter (0)")
plt.ylabel("No.of Client %")
upper_l=Non_defaulter_df1.groupby("NAME_INCOME_TYPE")["NAME_EDUCATION_TYPE"].value_counts().max()
total_l=Non_defaulter_df1.groupby("NAME_INCOME_TYPE")["NAME_EDUCATION_TYPE"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l,15) # Calling the user defined func to convert y label to %
plt.xticks(rotation=35)
plt.legend(loc=1)
        
plt.show()

#### Conclusion
- Approx 60 % of defaulter are from Working income type also a general is seen that secondary level education is doing more default.

# 🍔Merging of the data frame.

#### As our main aim is to find loan defaulter so it will be wise to merge the data frame via "Inner Join" as we will get know about the clients those who have previously applied and also currently want to apply for loan.

In [ ]:
dfmerge=pd.merge(left=df1,right=df,how="inner",on="SK_ID_CURR",suffixes=("_Current","_Previous"))
dfmerge.head()

In [ ]:
# Sanity Check

print("df1 ",df1.shape)
print("df ",df.shape)
print("dfmerge ",dfmerge.shape)

In [ ]:
dfmerge.info()

Let's just drop the columns which will not be the part of the analysis

In [ ]:
dfmerge.drop(columns=['EXT_SOURCE_2', 'EXT_SOURCE_3','FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3',
       'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6',
       'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9',
       'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_DOCUMENT_12',
       'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15',
       'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18',
       'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21',],inplace=True)

# 🌱Final Analysis

### 🎯TARGET Variable (Defaulter Data VS Non Defaulter) analysis

Yup good to go but before that we will bin few columns as per our requirements as we have not yet able to analyze based on age ,income so just give a shot.

In [ ]:
sns.boxplot(dfmerge.YEARS_BIRTH)
plt.show()

In [ ]:
dfmerge["Age_Bucket"]=pd.cut(x=dfmerge.YEARS_BIRTH,bins=[0,30,45,60,999],labels=["Young","Adult","Senior Adult","Senior"])

In [ ]:
sns.boxplot(dfmerge.AMT_INCOME_TOTAL)
plt.show()

#### Let's bucket the income in 3 category as outliers are already dealt with we have less than 3,50,000 amount income

Let's bucket client income now

In [ ]:
dfmerge["Income_Bucket"]=pd.cut(x=dfmerge.AMT_INCOME_TOTAL,bins=[0,150000,300000,100000000],labels=["Low","Medium ","High"])

In [ ]:
dfmerge.head()

In [ ]:
dfmerge.shape

### 💡Which loan type is in demand?

In [ ]:
plt.figure(figsize=[15,4])

plt.subplot(121)
sns.countplot(dfmerge.NAME_CONTRACT_TYPE_Current,hue=dfmerge.TARGET)
plt.title("Current application")
plt.ylabel("No.of Client %")

# Calling func converting y labels to %
upper_l=dfmerge.groupby("NAME_CONTRACT_TYPE_Current")["TARGET"].value_counts().max()
total_l=dfmerge.groupby("NAME_CONTRACT_TYPE_Current")["TARGET"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert yticks to percentage

plt.subplot(122)
sns.countplot(dfmerge.NAME_CONTRACT_TYPE_Previous,hue=dfmerge.TARGET)
plt.title("Previous application")
plt.ylabel("No.of Client %")

# Calling func converting y labels to %
upper_l=dfmerge.groupby("NAME_CONTRACT_TYPE_Previous")["TARGET"].value_counts().max()
total_l=dfmerge.groupby("NAME_CONTRACT_TYPE_Previous")["TARGET"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert yticks to percentage
plt.show()

#### Conclusion
- The demand for cash loanhas increased abruptly from 44% to 92% compare to the previous data set while there is no demand for consumer loan i.e., decreased to 0.

In [ ]:
#Storing Non defaulter data
Non_defaulter=dfmerge[dfmerge.TARGET==0]

#Storing defaulter data
Defaulter=dfmerge[dfmerge.TARGET==1]

# Univariate Analysis

### 💡Which Income bucket people have more  defaulter?

In [ ]:
plt.figure(figsize=[18,6])

plt.subplot(121)
sns.countplot(Defaulter.Income_Bucket) # Defaulter Income plot
plt.title("Defaulter (1)")
plt.ylabel("No. of client %")
convert_to_percentage(Defaulter.Income_Bucket)

plt.subplot(122)
sns.countplot(Non_defaulter.Income_Bucket) # Non Defaulter Income plot
plt.title("Non defaulter (0)")
plt.ylabel("No. of client %")
convert_to_percentage(Non_defaulter.Income_Bucket)


plt.show()

#### Conclusion
- The above graphs conclude that the person with  low and medium income tends to default more(about 47%) and this seems completely justified as   poor people miss the timely payment of installments.

### 💡Which Age group people have more defaulters?

In [ ]:
plt.figure(figsize=[18,6])

plt.subplot(121)
(Defaulter.Age_Bucket.value_counts(normalize=True)*100).plot.pie(autopct="%0.1f%%")
plt.title("Defaulter (1)")
plt.ylabel("")

plt.subplot(122)
(Non_defaulter.Age_Bucket.value_counts(normalize=True)*100).plot.pie(autopct="%0.1f%%")
plt.title("Non defaulter (0)")
plt.ylabel("")

plt.show()

#### Conclusion
- The graph states that the adult seems to have more default,it can be justified in a way that these group people are having more responsibilites.

### 💡What is the education level of most of the defaulter?

In [ ]:
plt.figure(figsize=[18,6])

plt.subplot(121)
sns.countplot(Defaulter.NAME_EDUCATION_TYPE)# Defaulters' education information
plt.xticks(rotation=30)
convert_to_percentage(Defaulter.NAME_EDUCATION_TYPE) # Calling the user defined func to convert yticks to percentage
plt.ylabel("No. of client %")
plt.title("Defaulter's education level")

plt.subplot(122)
sns.countplot(Defaulter.NAME_EDUCATION_TYPE,hue=Defaulter.CODE_GENDER) # Defaulters' education information Gender wise
upper_l=Defaulter.groupby("NAME_EDUCATION_TYPE")["CODE_GENDER"].value_counts().max()
total_l=Defaulter.groupby("NAME_EDUCATION_TYPE")["CODE_GENDER"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert yticks to percentage
plt.title("Defaulter's education level gender wise")
plt.ylabel("No. of client %")
plt.xticks(rotation=30)

plt.show()

### 💡What is the education level of most of the defaulter Income Bucket wise?

In [ ]:
# Defaulters' education information Gender wise
sns.countplot(Defaulter.NAME_EDUCATION_TYPE,hue=Defaulter.Income_Bucket)

upper_l=Defaulter.groupby("NAME_EDUCATION_TYPE")["Income_Bucket"].value_counts().max()
total_l=Defaulter.groupby("NAME_EDUCATION_TYPE")["Income_Bucket"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert yticks to percentage

plt.title("Defaulter's education level gender wise")
plt.ylabel("No. of client %")
plt.xticks(rotation=30)

plt.show()

#### Conclusion
- Most of the defaulter(80 %) are from Secondary level of education ,infact here also female count is high.
- Even from Secondary level 47 % are female.
- Defaulter's count decreases with increase in their education level.

### 💡What is the family status of most of the defaulter?

In [ ]:
plt.figure(figsize=[18,6])

plt.subplot(121)
Defaulter.NAME_FAMILY_STATUS.value_counts().plot.pie(autopct="%0.01f%%")
plt.title("Defaulter Family Status")
plt.ylabel("")

plt.subplot(122)
sns.countplot(Defaulter.NAME_FAMILY_STATUS,hue=Defaulter.Income_Bucket)


upper_l=Defaulter.groupby("NAME_FAMILY_STATUS")["Income_Bucket"].value_counts().max()
total_l=Defaulter.groupby("NAME_FAMILY_STATUS")["Income_Bucket"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) # Calling the user defined func to convert yticks to percentage
plt.title("Defaulter Family Status Income Bucket wise")
plt.ylabel("No. of client %")
plt.xticks(rotation=30)

plt.show()

#### Conclusion
- Married clients seem to do more default and most of them have either low or medium income .

### 💡Which region seems to be having more default case?

In [ ]:
plt.figure(figsize=[9,4])
 # Plot for defaulters
sns.distplot(Defaulter.REGION_POPULATION_RELATIVE,color="r",label="Defaulter (1)")
 # Plot for Non defaulters
sns.distplot(Non_defaulter.REGION_POPULATION_RELATIVE,color="c",label="Non defaulter (0)")
plt.legend()

plt.show()

#### Assumption 
- Cities have High population density
- Villages have low population density

#### Conclusion
- Though the pattern for both defaulter and non defaulter are same but point here is ,  maximium loan is taken by area with low population ie., by village people as cities' have high population density.

### 💡What is the term (months) of credit  pattern seen for most of the defaulter?

In [ ]:
plt.figure(figsize=[9,4])

sns.distplot(Defaulter.CNT_PAYMENT,color="r",label="Defaulter (1)") # Plot for defaulters

sns.distplot(Non_defaulter.CNT_PAYMENT,color="c",label="Non defaulter (0)") # Plot for Non defaulters

plt.legend()

plt.show()

#### Conclusion
- Maximum defaulters previous term of credit lies in the range 10 to 15 months and few are from 25 month tenure.


### 💡What can be said about GOODS LOAN?

In [ ]:
plt.figure(figsize=[15,4])

plt.subplot(221)
sns.distplot(dfmerge.AMT_GOODS_PRICE_Current,color="r",label="Current")
sns.distplot(dfmerge.AMT_GOODS_PRICE_Previous,color="c",label="Previous")
plt.legend()

plt.subplot(222)
sns.barplot(y=dfmerge.AMT_GOODS_PRICE_Current,color="r",label="Current")
sns.barplot(y=dfmerge.AMT_GOODS_PRICE_Previous,color="c",label="Previous")
plt.legend()

plt.show()

#### Conclusion
- The demand for goods loan is on increase both number and amount wise.

### 👀If the demand is high for goods loan then risk is also high for the same,let's analyze it further!!!! and 💡find the average spread ask from the defaulters.

In [ ]:
plt.figure(figsize=[15,4])

#Code for plot 1
plt.subplot(221)
sns.distplot(Defaulter.AMT_GOODS_PRICE_Current,color="r",label="Defaulter")
sns.distplot(Non_defaulter.AMT_GOODS_PRICE_Current,color="c",label="Non Defaulter")
plt.title("Current application")
plt.legend()

#Code for plot 2
plt.subplot(222)
sns.distplot(Defaulter.AMT_GOODS_PRICE_Previous,color="r",label="Defaulter")
sns.distplot(Non_defaulter.AMT_GOODS_PRICE_Previous,color="c",label="Non Defaulter")
plt.title("Previous application")
plt.legend()

plt.show()

#### Conclusion
- The defaulters ask this time,mainly for GOODS PRICE ranges from 2 to 5 lakh.

### 💡What is the spread of credit amount given by bank to it's client in compare to the previous credit?

In [ ]:
plt.figure(figsize=[15,4])

plt.subplot(221)
sns.distplot(Defaulter.AMT_CREDIT_Current,color="r",label="Defaulter")
sns.distplot(Non_defaulter.AMT_CREDIT_Current,color="c",label="Non Defaulter")
plt.title("Current application")
plt.legend()

plt.subplot(222)
sns.distplot(Defaulter.AMT_CREDIT_Previous,color="r",label="Defaulter")
sns.distplot(Non_defaulter.AMT_CREDIT_Previous,color="c",label="Non Defaulter")
plt.title("Previous application")
plt.legend()


plt.show()

#### Conclusion
- The distribution of current credit amount has increased greatly.
- Earlier maximum credit given was around 5 lakh but this time it is around 17.5 lakh.
- Earlier defaulters' credit amount were less than 1 lakh but this time most of the defaulters' lies in the range of 0-7.5 lakh

### 💡What is the effect of the interest rate on the defaulters?

In [ ]:
sns.countplot(Defaulter.NAME_YIELD_GROUP,hue=Defaulter.Income_Bucket,order=Defaulter.NAME_YIELD_GROUP.value_counts().index)
plt.xticks(rotation=30)
upper_l=Defaulter.groupby("NAME_YIELD_GROUP")["Income_Bucket"].value_counts().max()
total_l=Defaulter.groupby("NAME_YIELD_GROUP")["Income_Bucket"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l)
plt.ylabel("No. of client %")
plt.title("Defaulter")
plt.show()

### Note
- I have not imputed the XNA in the above plot as it's meaning is not clearly mentioned in the data dictionary ,also it is huge in number so imputing it without having it's proper information will not bring justic to my analysis.  

#### Conclusion
- The loan with greater interest seems to default more so we can draw a conclusion that greater the interest rate higher is the risk

## Bivariate Analysis

### 💡Which Income Type People seems to cause more default?

In [ ]:
plt.figure(figsize=[15,4])

plt.subplot(121)
sns.barplot(Defaulter.NAME_INCOME_TYPE,Defaulter.AMT_INCOME_TOTAL,ci=None)
plt.xticks(rotation=90)
plt.title("Defaulter")

plt.subplot(122)
sns.countplot(Defaulter.NAME_INCOME_TYPE,hue=Defaulter.Income_Bucket)
plt.title("Defaulter Income Type")
plt.ylabel("No. of client %")

upper_l=Defaulter.groupby("NAME_INCOME_TYPE")["Income_Bucket"].value_counts().max()
total_l=Defaulter.groupby("NAME_INCOME_TYPE")["Income_Bucket"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l)
plt.xticks(rotation=90)

plt.show()

#### Conclusion
- Persons with low income bucket seems to do more default
- Working  category takes major loan.

## 💡Which Income type based on age seems to default more?

In [ ]:
plt.figure(figsize=[15,4])

plt.subplot(121)
sns.countplot(Defaulter.NAME_INCOME_TYPE,hue=Defaulter.Age_Bucket)
plt.title("Defaulter")
plt.ylabel("No. of client %")

upper_l=Defaulter.groupby("NAME_INCOME_TYPE")["Age_Bucket"].value_counts().max()
total_l=Defaulter.groupby("NAME_INCOME_TYPE")["Age_Bucket"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l)

plt.xticks(rotation=35)
plt.legend(loc=1)

plt.subplot(122)
sns.barplot(Defaulter.NAME_INCOME_TYPE,Defaulter.AMT_INCOME_TOTAL,hue=Defaulter.Age_Bucket,ci=None)
plt.title("Defaulter")
plt.xticks(rotation=35)
plt.legend(loc=1)

plt.show()

#### Conclusion
- Working Adults seem to have more default case .
- Average income of Commercial associate are  compartively greater than others.
- Among defaulters senior are having higher average income.

### 💡 For how long  defaulters have been earning ?

In [ ]:
sns.scatterplot(Defaulter.AMT_INCOME_TOTAL,Defaulter.DAYS_EMPLOYED,hue=Defaulter.CODE_GENDER)
plt.legend(bbox_to_anchor=(1,1))
plt.show()

#### Conclusion
- Most of the defaulters income total is less than 2.5 lakh and most of them are working for about 10 years.
- Most of the female defaulter has income less than 2 lakh.

### 💡 Occupation with their Income type Pattern

In [ ]:
plt.figure(figsize=[15,6])

sns.countplot(Defaulter.NAME_INCOME_TYPE,hue=Defaulter.OCCUPATION_TYPE)
# Calling the user defined func to convert yticks to percentage
upper_l=Defaulter.groupby("NAME_INCOME_TYPE")["OCCUPATION_TYPE"].value_counts().max()
total_l=Defaulter.groupby("NAME_INCOME_TYPE")["OCCUPATION_TYPE"].value_counts().sum()
convert_to_percentage_hue(upper_l,total_l) 

plt.ylabel("No. of client %")
plt.xticks(rotation=30)
plt.legend(bbox_to_anchor=(1,1))
plt.title("Defaulter's Occupation with their Income type")
plt.show()

#### Conclusion
- Working defaulter are the majority and among them Laborers are the higher defaulters.
- Pensioner also consists of 12 % among defaulters.

### Top 10 Defaulter Correlation

In [ ]:
top10_Corr(Defaulter) # top 10 defaulter correlation

### Non Defaulter Correlation

In [ ]:
top10_Corr(Non_defaulter) # Top 10 Non defaulter correlation

#### Conclusion
- In all the correlations there seems to be a strong relation between good price and amount credit.

# Final Conclusion (Defaulter Pattern )

Bank should pay extra attention to following criteria whlie giving loan -
- Loan Type - Cash loan
- Gender - "Female" clients 
- Eucation Type - "Secondary level" of education (Higher the  Education lesser the  default)
- Occupation Type - "WORKING" and also little focus on "Commercial associates/Pensioner" 
- INCOME Type - "LABOURERS"
- Income Bucket - "LOW/MEDIUM"
- Age Bucket - "Adult"
- Interest charged by bank - Higher the rate charged greater the risk
- Credit Amount - Most of the defaulters' lies in the range of 0-7.5 lakh
- Family status - "Married"
- Organization Type - Bussiness Type 3 /Self Employeed

